# DC Data Blend

In [1]:
#Import packages
import os
import pandas as pd 
import numpy as np
import datetime as dt

# Bike Data

In [ ]:
#Import bike trip data
path = r'path'

filenames = [file for file in os.listdir('.') if file.startswith('Divvy')] #Create list of files

dfs = [] #Empty DF

for file in filenames:
    dfs.append(pd.read_csv(file)) #Read files in and add to dfs
    
DC_df = pd.concat(dfs, ignore_index = True) #concatenate dfs

#Drop uneeded columns
to_drop = ['bikeid', 'birthday', 'birthyear', 'from_station_id', 'from_station_name', 'gender', 'to_station_id', 'to_station_name', 'trip_id']

for col in list(DC_df):
    if col in to_drop:
        DC_df = DC_df.drop(col, 1)

#Add dummy for usertype and drop
user_dummy = pd.get_dummies(DC_df['usertype'])
DC_df = pd.concat([DC_df, user_dummy], axis = 1)
DC_df = DC_df.drop(['usertype'], axis = 1)

#Convert starttime to DateTime (Long Process)
DC_df.starttime = pd.to_datetime(DC_df.starttime)
DC_df.stoptime = pd.to_datetime(DC_df.stoptime)

#Calculate Duration
DC_df['Duration'] = DC_df.stoptime - DC_df.starttime
DC_df = DC_df.drop(['tripduration'], axis = 1)
time = pd.DatetimeIndex(DC_df.Duration)
DC_df.Duration = time.hour * 60 + time.minute

#Set index to starttime
DC_time_sort_df = DC_df.reset_index().set_index('starttime')
DC_time_sort_df = DC_time_sort_df.drop(DC_time_sort_df.columns[0], axis = 1) #Drop index column
DC_time_sort_df['Total_Count'] = float(1) #Add counter for total count

#Create stats dict for column stats
f = {'Duration': ('mean', 'min', 'max'), 'Customer': 'sum', 'Dependent': 'sum', 'Subscriber': 'sum', 'Total_Count': 'sum'}

#Aggregate data by day
DC_by_day = DC_time_sort_df.groupby(pd.TimeGrouper("D")).agg(f) #Groupby day and apply agg(f)
DC_by_day_df = pd.DataFrame(DC_by_day) #Create aggregated DF
DC_by_day_df = DC_by_day_df.ix[4:] #Drop first 4 days (don't match with weather)

print(DC_by_day_df.shape)
DC_by_day_df.head()

# Weather Data

In [ ]:
#Import weather data
weather_df = pd.read_csv('DC_Weather.csv')
weather_df = weather_df.fillna('')
print(weather_df.shape)
weather_df.head()

#Drop STATION and STATION_NAME
weather_df = weather_df.drop(['STATION', 'STATION_NAME'], axis = 1)

#Replace 'T' with 0 fpr precip
weather_df.HOURLYPrecip.replace('T', 0, inplace = True)

#Convert DATE to DateTime
weather_df.DATE= pd.to_datetime(weather_df.DATE)

#Index DATE and convert columns to_numeric for aggregation
weather_time_sort_df = weather_df.reset_index().set_index('DATE')
weather_time_sort_df = weather_time_sort_df.drop(weather_time_sort_df.columns[0], axis = 1)
weather_time_sort_df = weather_time_sort_df.convert_objects(convert_numeric = True)
weather_time_sort_df = weather_time_sort_df.drop(['REPORTTPYE', 'HOURLYSKYCONDITIONS', 'HOURLYPRSENTWEATHERTYPE'], axis = 1)

#Create stats dict for column stats
f = {}
stats = ['sum', 'mean', 'max', 'min']

for col in list(weather_time_sort_df):
    f[col] = stats

weather_by_day = weather_time_sort_df.groupby(pd.TimeGrouper("D")).agg(f)
weather_by_day_df = pd.DataFrame(weather_by_day)

print(weather_by_day_df.shape)
weather_by_day_df.head()

# Final Dataset

In [ ]:
#Merge Datasets
merged = pd.concat([DC_by_day_df, weather_by_day_df], join = 'inner', axis = 1)
merged.to_csv('DC_Blended(new).csv') #Create .csv file

print(merged.shape)
merged.head()